In [1]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [2]:
import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(528,528), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [4]:
COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

Running on 1 replicas


In [5]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)

load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

# Get the multi-labels
label_cols = sub_df.columns[1:]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[6], IMSIZE[6]))
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

In [ ]:
#../input/ranzcr-efficientnet-tpu-training/model.h5

In [6]:
with strategy.scope():
    model = tf.keras.models.load_model('../input/efficientnetb6-with-fine-tuned-training-tpu/EfficientNetB6_FineTuned.h5')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b6 (Functional) (None, 17, 17, 2304)      40960136  
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 17, 17, 2304)      0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)                25355     
Total params: 40,985,491
Trainable params: 40,761,059
Non-trainable params: 224,432
_________________________________________________________________


In [7]:
sub_df[label_cols] = model.predict(dtest, verbose=1)
sub_df.to_csv('submission.csv', index=False)

sub_df.head()

224/224 [==============================] - 254s 1s/step


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.005576,8.643032e-01,1.797117e-01,7.561186e-04,1.009264e-03,2.022859e-02,9.298095e-01,0.059499,0.161583,0.938906,9.999917e-01
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000001,3.835015e-07,6.008776e-10,5.629590e-08,1.391413e-07,1.284343e-08,1.583386e-09,0.000245,0.001511,0.999042,3.053007e-07
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000004,3.661080e-07,1.187400e-08,1.271503e-06,2.608744e-06,9.278492e-08,2.697200e-08,0.001823,0.732819,0.453641,1.056061e-06
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.000042,4.243460e-02,6.722198e-02,2.020283e-03,4.502008e-05,9.969656e-01,1.286229e-04,0.000755,0.000570,0.998949,1.680040e-02
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000246,4.541046e-04,4.070242e-05,1.296829e-05,2.855204e-05,2.484462e-06,1.116362e-03,0.002565,0.150246,0.931262,1.858199e-06
